In [2]:
import sys
import os
import pandas as pd
import requests
import matplotlib.pyplot as plt

In [6]:
sys.path.append('/Users/mm40108/projects/datadays17/olfactory_protein_viz/src/')
from data.ABA_utils import get_products, get_genes, get_genes_from_aliases, get_experiments_and_images
from data.ABA_image_utils import get_image, get_organism_image_params, save_image_locally

THIS_DIR = "/Users/mm40108/projects/datadays17/olfactory_protein_viz/src/data"
DATA_DIR = "/Users/mm40108/projects/datadays17/olfactory_protein_viz/data"

In [7]:
# 1 user selects product
all_products = get_products(atlases_only=True)
all_products.head()

,abbreviation,description,id,name,product_name_facet,resource,species,species_name_facet,tags
0,HumanASD,High-resolution ISH data detailing gene expres...,26,Human Brain ISH Autism Study,3312816226,Allen Human Brain Atlas,Human,1688468209,None
3,ConnRef,Reference datasets created for the Allen Mouse...,7,Mouse Connectivity Reference Data,3548889432,Allen Mouse Brain Connectivity Atlas,Mouse,1861523945,None
5,HumanSubCtx,High-resolution ISH data detailing the express...,10,Human Brain ISH SubCortex Study,4053002594,Allen Human Brain Atlas,Human,1688468209,None
6,DevHumanISH,High-resolution ISH data detailing gene expres...,22,Developing Human Brain ISH,131071957,BrainSpan Atlas of the Developing Human Brain,Human,1688468209,None
7,NhpMMA,Genome-wide microarray data profiling gene exp...,28,Non-Human Primate Macrodissection Microarray,2470590257,NIH Blueprint Non-Human Primate (NHP) Atlas,NHP,3599691041,None


In [8]:
# these are all available products:
all_products[['name', 'abbreviation']].values.tolist()

[['Human Brain ISH Autism Study', 'HumanASD'],
 ['Mouse Connectivity Reference Data', 'ConnRef'],
 ['Human Brain ISH SubCortex Study', 'HumanSubCtx'],
 ['Developing Human Brain ISH', 'DevHumanISH'],
 ['Non-Human Primate Macrodissection Microarray', 'NhpMMA'],
 ['Mouse Connectivity Vertical Retina', 'ConnRetinaVertical'],
 ['Human Brain ISH Schizophrenia Study', 'HumanSZ'],
 ['Human Brain Microarray', 'HumanMA'],
 ['Non-Human Primate Microdissection Microarray', 'NhpLMA'],
 ['Non-Human Primate Molecular Map', 'NhpMM'],
 ['Human Brain ISH Neurotransmitter Study', 'HumanNT'],
 ['Developing Mouse Brain Reference Data', 'DevMouseRef'],
 ['Human Brain ISH Cortex Study', 'HumanCtx'],
 ['Mouse Connectivity anterograde Cre-defined EGFP', 'ConnAntCreEGFP'],
 ['Non-Human Primate ISH', 'NhpISH'],
 ['Developing Human Prenatal LMD Microarray', 'DevHumanMA'],
 ['Mouse connectivity target-defined anterograde projectome (associational cortical areas)',
  'ConnAntTargetAssocCort'],
 ['Mouse Connectivity

In [10]:
# Let's say the user picks:['Developing Mouse Brain', 'DevMouse'], we take the abbreviation
product = 'DevMouse'

In [11]:
# 2 user selects genes available in this product line
genes = get_genes(product)
genes.head()

	attempting query: http://api.brain-map.org/api/v2/data/Gene/query.json?criteria=model::Gene,rma::criteria,products[abbreviation$eq'DevMouse']&num_rows=all
	response: (2107, 16) 
	 Index(['acronym', 'alias_tags', 'chromosome_id', 'ensembl_id', 'entrez_id',
       'genomic_reference_update_id', 'homologene_id', 'gene_id',
       'legacy_ensembl_gene_id', 'name', 'organism_id', 'original_name',
       'original_symbol', 'reference_genome_id', 'sphinx_id',
       'version_status'],
      dtype='object')


,acronym,alias_tags,chromosome_id,ensembl_id,entrez_id,genomic_reference_update_id,homologene_id,gene_id,legacy_ensembl_gene_id,name,organism_id,original_name,original_symbol,reference_genome_id,sphinx_id,version_status
0,Slc1a6,EAAT4,35.0,None,20513.0,491928275.0,21055.0,20275,None,solute carrier family 1 (high affinity asparta...,2,solute carrier family 1 (high affinity asparta...,Slc1a6,None,102670,no change
1,Lhx3,Lim3 mLim-3 mLIM3 P-LIM,47.0,None,16871.0,491928275.0,7814.0,16642,None,LIM homeobox protein 3,2,LIM homeobox protein 3,Lhx3,None,91840,no change
2,Dlx2,AW121999 Dlx-2 Tes-1,47.0,None,13392.0,491928275.0,3244.0,13171,None,distal-less homeobox 2,2,distal-less homeobox 2,Dlx2,None,92911,no change
3,Ncam2,Ncam-2 Ocam RNCAM,42.0,None,17968.0,491928275.0,3336.0,17735,None,neural cell adhesion molecule 2,2,neural cell adhesion molecule 2,Ncam2,None,89989,no change
4,Hcrtr1,MGC141357 Ox1r,49.0,None,230777.0,491928275.0,37492.0,87114,None,hypocretin (orexin) receptor 1,2,hypocretin (orexin) receptor 1,Hcrtr1,None,99831,no change


In [16]:
# user selects genes of interest
genes_of_interest = ['Robo2', 'Cdh5', 'Bmpr2']
our_genes = genes[genes.acronym.isin(genes_of_interest)]
our_gene_ids = our_genes.gene_id.values.tolist()
our_genes.head()

,acronym,alias_tags,chromosome_id,ensembl_id,entrez_id,genomic_reference_update_id,homologene_id,gene_id,legacy_ensembl_gene_id,name,organism_id,original_name,original_symbol,reference_genome_id,sphinx_id,version_status
851,Bmpr2,2610024H22Rik AL117858 AW546137 BB189135 BMP-2...,34.0,None,12168.0,491928275.0,929.0,11954,None,"bone morphogenetic protein receptor, type II (...",2,"bone morphogenic protein receptor, type II (se...",Bmpr2,None,358483,updated
1111,Robo2,2600013A04Rik 9430089E08Rik BB097918 D230004I2...,42.0,None,268902.0,491928275.0,43188.0,92611,None,roundabout homolog 2 (Drosophila),2,roundabout homolog 2 (Drosophila),Robo2,None,103280,no change
1447,Cdh5,7B4 AA408225 Cd144 CD144 Vec VEC VEcad VE-Cad ...,53.0,None,12562.0,491928275.0,1359.0,12347,None,cadherin 5,2,cadherin 5,Cdh5,None,101182,no change


In [19]:
# get information on SectionDataSets and SectionImges
exp_data, img_data = get_experiments_and_images(genes_of_interest, organism='DevMouse')

	attempting query: http://api.brain-map.org/api/v2/data/Gene/query.json?criteria=model::Gene,rma::criteria,products[abbreviation$eq'DevMouse']&num_rows=all
	response: (2107, 16) 
	 Index(['acronym', 'alias_tags', 'chromosome_id', 'ensembl_id', 'entrez_id',
       'genomic_reference_update_id', 'homologene_id', 'gene_id',
       'legacy_ensembl_gene_id', 'name', 'organism_id', 'original_name',
       'original_symbol', 'reference_genome_id', 'sphinx_id',
       'version_status'],
      dtype='object')
['Bmpr2', '2610024H22Rik', 'AL117858', 'AW546137', 'BB189135', 'BMP-2', 'BMPR-2', 'BMPRII', 'BMPR-II', 'BRK-3', 'Gm20272', 'Bmpr2']
found  Bmpr2  has id  11954
['Robo2', '2600013A04Rik', '9430089E08Rik', 'BB097918', 'D230004I22Rik', 'mKIAA1568', 'Robo2']
found  Robo2  has id  92611
['Cdh5', '7B4', 'AA408225', 'Cd144', 'CD144', 'Vec', 'VEC', 'VEcad', 'VE-Cad', 'VECD', 'Cdh5']
found  Cdh5  has id  12347

results of searching for  ['Robo2', 'Cdh5', 'Bmpr2'] 
	 {'Bmpr2': 11954, 'Robo2': 92611,

	response: (16, 25) 
	 Index(['annotated', 'axes', 'bits_per_component', 'data_set_id',
       'expression_path', 'failed', 'height', 'id', 'image_height',
       'image_type', 'image_width', 'isi_experiment_id', 'lims1_id',
       'number_of_components', 'ophys_experiment_id', 'path',
       'projection_function', 'resolution', 'section_number', 'specimen_id',
       'structure_id', 'tier_count', 'width', 'x', 'y'],
      dtype='object')
getting all images for experiment_id 100045469
	attempting query: http://api.brain-map.org/api/v2/data/query.json?criteria=model::SectionImage,rma::criteria,[data_set_id$eq100045469]&num_rows=all
	response: (18, 25) 
	 Index(['annotated', 'axes', 'bits_per_component', 'data_set_id',
       'expression_path', 'failed', 'height', 'id', 'image_height',
       'image_type', 'image_width', 'isi_experiment_id', 'lims1_id',
       'number_of_components', 'ophys_experiment_id', 'path',
       'projection_function', 'resolution', 'section_number', 'specimen_id

	response: (20, 25) 
	 Index(['annotated', 'axes', 'bits_per_component', 'data_set_id',
       'expression_path', 'failed', 'height', 'id', 'image_height',
       'image_type', 'image_width', 'isi_experiment_id', 'lims1_id',
       'number_of_components', 'ophys_experiment_id', 'path',
       'projection_function', 'resolution', 'section_number', 'specimen_id',
       'structure_id', 'tier_count', 'width', 'x', 'y'],
      dtype='object')
getting all images for experiment_id 100056389
	attempting query: http://api.brain-map.org/api/v2/data/query.json?criteria=model::SectionImage,rma::criteria,[data_set_id$eq100056389]&num_rows=all
	response: (17, 25) 
	 Index(['annotated', 'axes', 'bits_per_component', 'data_set_id',
       'expression_path', 'failed', 'height', 'id', 'image_height',
       'image_type', 'image_width', 'isi_experiment_id', 'lims1_id',
       'number_of_components', 'ophys_experiment_id', 'path',
       'projection_function', 'resolution', 'section_number', 'specimen_id

In [20]:
exp_data.head()

,blue_channel,delegate,expression,failed,failed_facet,green_channel,experiment_id,name,plane_of_section_id,qc_date,red_channel,reference_space_id,rnaseq_design_id,section_thickness,specimen_id,sphinx_id,storage_directory,weight,gene_id
0,None,True,True,False,734881840,None,100046444,None,2,2009-07-28T16:18:33Z,None,3,None,20.0,4908,148630,/external/devmouse/prod165/image_series_100046...,5200,11954
1,None,True,True,False,734881840,None,100042306,None,2,2009-02-18T14:28:00Z,None,8,None,25.0,4797,83381,/external/devmouse/prod165/image_series_100042...,5200,11954
2,None,True,True,False,734881840,None,69529382,None,2,2009-05-02T22:47:10Z,None,10,None,25.0,69486038,34806,/external/aibssan/production32/prod330/image_s...,5470,11954
3,None,True,True,False,734881840,None,100057140,None,2,2009-05-04T16:12:12Z,None,5,None,20.0,6226,43380,/external/devmouse/prod170/image_series_100057...,5200,11954
4,None,True,True,False,734881840,None,100057296,None,2,2009-05-06T15:44:29Z,None,6,None,20.0,6403,58993,/external/devmouse/prod170/image_series_100057...,5200,11954


In [21]:
img_data.head()

,annotated,axes,bits_per_component,data_set_id,expression_path,failed,height,id,image_height,image_type,...,projection_function,resolution,section_number,specimen_id,structure_id,tier_count,width,x,y,experiment_id
0,False,None,8,100046444,/external/devmouse/prod41/0500070414/050007041...,False,15008,100938255,16144,Primary,...,None,0.99,54,None,None,9,9488,2320,304,100046444
1,False,None,8,100046444,/external/devmouse/prod43/0500069837/050006983...,False,14816,100938261,16672,Primary,...,None,0.99,42,None,None,9,11040,24944,1104,100046444
2,False,None,8,100046444,/external/devmouse/prod28/0500071135/050007113...,False,14176,100938266,14352,Primary,...,None,0.99,24,None,None,9,10896,37680,0,100046444
3,False,None,8,100046444,/external/devmouse/prod43/0500069837/050006983...,False,14240,100938259,16672,Primary,...,None,0.99,30,None,None,9,9392,992,1856,100046444
4,False,None,8,100046444,/external/devmouse/prod41/0500070117/050007011...,False,15104,100938252,16656,Primary,...,None,0.99,84,None,None,9,9872,15056,1536,100046444
